# Parallel calculation in Julia

## A famous example

Can the codes below be paralleled?

```fortran
do i = 1, 1000000
    s = s + x[i]
end
```

Above program is buggy in the first place, where `s` was not initialized:

```fortran
s = 0   ! OOPS
do i = 1, 1000000
    s = s + x[i]
end
```

Writing program this way makes it very hard to parallelize it, even (not necessary right in language details)

```fortran
s = 0
parallel do i = 1, 1000000
    atom s = s + x[i]
end
```

## A few common problems in parallel programming

- Too many threads
  - Distributing overhead
  - Physical versus virtual CPU/threads
- Data race
  - Thread-safe
- Others
  - Load balancing
  - Dead locks

This example was mentioned in Guy L. Steele Jr.'s talk [How to Think about Parallel Programming: Not!](https://www.infoq.com/presentations/Thinking-Parallel-Programming/), and his Google TechTalk [Four Solutions to a Trivial Problem](https://www.youtube.com/watch?v=ftcIcn8AmSY).  Guy L. Steele Jr. believes that it should not be the programmer’s job to think about parallelism, but languages should provide ways to transparently run tasks in parallel. This requires a new approach in building languages supporting algorithms built on independence and build-and-conquer principles rather than on linear decomposition of problems.

<!-- Algebraic properties needed: Associative, Cummutative, Idempotent, Identity, Zero. Thatis grouping, order, duplicates don't matter. One value or all the rest value can be ignored. -->

Guy's talk inspired [several julia packages](https://juliafolds.github.io/data-parallelism/tutorials/quick-introduction/), e.g., `Folds`, `FLoops`, `Transducers`. You are encouraged to go through this short introduction. 

## Parallel using `Distributed`, and `Threads`

We use here the famous $\pi$ estimating with Monte Carlo methods. We draw random points in sqaure defined by (0, 0), (0, 1), (1, 1), (1, 0), the proportion of points within the quarter unit circle in the first quadrant is $\frac{\pi}4$.

The packages to be used today is as below.

In [ ]:
using Distributed, Folds, Plots, BenchmarkTools, LinearAlgebra # and Threads

In [ ]:
# A plot demonstration

circley(x) = sqrt(1 - x^2)
plot(circley, 0, 1, leg=false, size = (400, 400))

x1 = []
y1 = []
x2 = []
y2 = []

for _ in 1:1000
    x = rand()
    y = rand()
    if x^2 + y^2 <= 1
        push!(x1, x)
        push!(y1, y)
    else
        push!(x2, x)
        push!(y2, y)
    end
end

scatter!(x1, y1, leg=false)
scatter!(x2, y2, leg=false)

In [ ]:
# Function
"""
    function mcπ(n)
Runs a simple Monte Carlo method to estimate pi with `n` samples.
"""
function mcπ(n)
    s = 0
    for _ in 1:n
        x, y = rand(), rand()
        x^2 + y^2 <= 1 && (s += 1)
        # or 
        # s += (x^2 + y^2) <= 1
    end
    return 4s/n
end

In [ ]:
mcπ(1000)

### Using `Distributed`,

```julia
julia -p 4
nprocs() # -> 5
nworkders() # -> 4
addprocs(2)
nprocs() # -> 7
nworkders() # -> 6
```

In [ ]:
addprocs(3)

In [ ]:
@btime mcπ(1_000_000_000)

In [ ]:
@everywhere include("mcpi.jl")

In [ ]:
using Statistics

parallelpi(N) = mean(pmap(n -> mcπ(n), [N/nworkers() for i in 1:nworkers()]))


In [ ]:
@btime parallelpi(1_000_000_000)

### Using `Base.Threads`

```julia
JULIA_NUM_THREADS=4 julia # bash command

nthreads()
```

In [ ]:
Threads.nthreads()

In [ ]:
# A shared array example
A = zeros(Threads.nthreads())
Threads.@threads for i in 1:Threads.nthreads()
    A[i] = Threads.threadid()
end
A

In [ ]:
nt = Threads.nthreads()
@btime Threads.@threads for i in 1:nt
    A[i] = mcπ(1_000_000_000 ÷ nt)
end

In [ ]:
mean(A)

A 

## Using established libs.

- BLAS
- LAPACK

MKL is a commercial (free available), widely used library.

In [ ]:
BLAS.get_num_threads()
BLAS.set_num_threads(1)
n = 8000
matA = rand(n, n)
matB = rand(n, n)
matC = similar(matA)

@btime BLAS.gemm!('N', 'N', 1., matA, matB, 0., matC) # C = αA * B + βC
BLAS.set_num_threads(2)
@btime BLAS.gemm!('N', 'N', 1., matA, matB, 0., matC) # C = αA * B + βC

# peakflops(n)  # to check your computer's ability.

## What's next?

- MPI?
- GPU